# 랭체인, Langchain

언어 모델에 기반한 AI 애플리케이션을 쉽게 개발할 수 있도록 도와주는 프레임워크
- 언어모델 API를 사용하기 위하 모든 코드를 직접 작성해야하는 작업을 간소화할 수 있는 다양한 도구와 모듈 제공
- 복잡한 애플리케이션을 개발할 때 미리 구축된 모듈을 활용
- 언어 모델 교체 용이 
    - LLM 서비스마다 API가 달라서 언어 모델 교체 시 코드 전반을 수정해야 함
    - 랭체인을 사용하면 언어 모델 언언 부분만 수정하면 됨

### 개발 환경 구축 (가상 환경 LangchainEnv, python=3.12)
- pip install ipykernel
- pip install langchain
- pip install langchain-openai

In [7]:
# llm 모델 생성
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스

server_endpoint = "http://127.0.0.1:1234/v1"
model = "google/gemma-3-12b"

llm = ChatOpenAI(
    base_url=server_endpoint,
    api_key="not_needed",
    model=model
)

## 채팅 관련 Messages : HumanMessage, AIMessage, SystemMessage
https://python.langchain.com/api_reference/core/messages.html

In [9]:
from langchain_core.messages import HumanMessage
result=llm.invoke([HumanMessage(content="안녕 난 조시현이야 ")])
print(result.content)

안녕하세요, 조시현님! 만나서 반갑습니다. 무엇을 도와드릴까요? 😊 궁금한 점이 있거나 이야기하고 싶은 내용이 있다면 편하게 말씀해주세요.


In [13]:
result=llm.invoke([HumanMessage("너는 누구니?")])
print(result.content)

저는 Google에서 훈련한 대규모 언어 모델입니다.


# 멀티턴 : Message History 기능

### InMemoryChatMessageHistory
* 메모리 내에서 메시지를 리스트 형태로 보관

### RunnableWithMessageHistory
모델을 생성할 때 대화 기록을 함께 전달할 수 있도록 하는 클래스
model 대신 RunnableWithMessageHistory의 인스턴스 실행

In [14]:
# 메모리에 대화 기록을 저장하는 클래스
from langchain_core.chat_history import InMemoryChatMessageHistory

# 메시지 기록을 활용해 실행 가능한 wrapper 클래스
from langchain_core.runnables import RunnableWithMessageHistory

# 세션별 대화 기록을 저장할 딕셔너리 (실제 운영 환경에서는 DB 활용)
history_store={}

# 세션 ID에 따라 대화 기록(InMemoryChatMessageHistory 객체)을 가져오는 함수 
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in history_store:
        # 메모리에 대화 기록을 저장하는 객체 생성
        history_store[session_id]=InMemoryChatMessageHistory()
    # 해당 세션의 대화 기록을 반환
    return history_store[session_id]

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history=RunnableWithMessageHistory(llm,get_session_history)

In [20]:
config =  {"configurable": {"session_id": "bbb"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름은 조상원이야")],
    config=config,
)

print(response.content)

안녕하세요, 조상원 님! 만나서 반갑습니다. 😊 어떤 도움이 필요하신가요? 궁금한 점이 있거나 이야기하고 싶으신 내용이 있다면 편하게 말씀해주세요.


In [21]:
config =  {"configurable": {"session_id": "bbb"}}

response = with_message_history.invoke(
    [HumanMessage(content="내이름은 뭘까?")],
    config=config,
)

print(response.content)

당신(혹은 당신의 프로필)에 따르면, 당신의 이름은 조상원입니다! 😊 

제가 기억하는 한, 이 대화에서 당신은 "내 이름은 조상원이야"라고 말씀하셨어요. 혹시 다른 이름을 말씀하시는 건가요?


In [17]:
result=llm.invoke([HumanMessage("너는 누구니?")])
print(result.content)

저는 Google에서 훈련한 대규모 언어 모델입니다.


### Stream 방식으로 출력하기
invoke() 대신 stream() 사용 -> 반복문으로 답변이 생성되는 대로 데이터를 계속 출력

In [23]:
config =  {"configurable": {"session_id": "ccc"}}

for response in with_message_history.stream([HumanMessage(content="한국에 대해서 알려줘 ")],config=config):
    print(response.content, end="")

## 한국에 대해 궁금하신가요? 😊

한국(Republic of Korea, 대한민국)은 동아시아의 한반도 남부에 위치한 나라입니다. 다채로운 매력을 가진 한국에 대한 다양한 정보를 알려드릴게요!

**1. 기본 정보:**

*   **수도:** 서울 (Seoul)
*   **언어:** 한국어 (Korean)
*   **인구:** 약 5,175만 명 (2023년 기준)
*   **면적:** 10만 4869㎢ (남한 전체 면적)
*   **통화:** 대한민국 원 (KRW)
*   **종교:** 불교, 기독교, 천주교 등 다양한 종교를 믿는 사람들이 있습니다.

**2. 역사 & 문화:**

*   **고대사:** 한반도에는 구석기 시대부터 사람이 살았습니다. 고조선, 삼국시대(고구려, 백제, 신라)를 거쳐 고려와 조선 시대를 지나왔습니다.
*   **근현대사:** 일제강점기를 겪은 후 해방되었지만 한국전쟁으로 큰 피해를 입었습니다. 이후 경제 성장을 이루어 '한강의 기적'이라고 불리는 눈부신 발전을 경험했습니다.
*   **문화:**
    *   **음식:** 김치, 비빔밥, 불고기, 삼겹살 등 매콤하고 짭짤한 맛이 특징입니다. 다양한 해산물 요리도 유명합니다.
    *   **전통 의상:** 한복은 아름다운 색감과 우아한 디자인으로 한국의 전통을 대표하는 의상입니다.
    *   **음악:** K-Pop은 전 세계적으로 인기를 얻고 있으며, 국악은 한국 고유의 아름다운 음악입니다.
    *   **예술:** 한국화, 서예, 민속 공예 등 다양한 예술 분야가 발달했습니다.

**3. 경제 & 산업:**

*   **주요 산업:** 반도체, 자동차, 조선, 전자제품, 석유화학 등 제조업이 강세를 보입니다.
*   **IT 강국:** 세계 최고 수준의 인터넷 환경과 스마트폰 보급률을 자랑합니다.
*   **서비스 산업:** 관광, 금융, 엔터테인먼트 산업도 빠르게 성장하고 있습니다.

**4. 여행 & 관광:**

*   **주요 관광지:**
    *   

# ChatBot 구현
1. llm 모델 클라이언트 생성
2. 랭체인 메모리 객체 생성
3. 대화

In [1]:
# 1. llm 모델 클라이언트 생성
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스

server_endpoint = "http://127.0.0.1:1234/v1"
model = "google/gemma-3-12b"

llm = ChatOpenAI(
    base_url=server_endpoint,
    api_key="not_needed",
    model=model
)

In [2]:
# 2. 랭체인 메모리 객체 생성
# 메모리에 대화 기록을 저장하는 클래스
from langchain_core.chat_history import InMemoryChatMessageHistory
# 메시지 기록을 활용해 실행 가능한 wrapper 클래스
from langchain_core.runnables import RunnableWithMessageHistory
# 세션별 대화 기록을 저장할 딕셔너리 (실제 운영 환경에서는 DB 활용용)
from langchain_core.messages import HumanMessage
history_store = {}
# 세션 ID에 따라 대화 기록(InMemoryChatMessageHistory 객체)을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in history_store:
        # 메모리에 대화 기록을 저장하는 객체 생성
        history_store[session_id] = InMemoryChatMessageHistory()
        history_store[session_id].add_message(
            SystemMessage(content="너는 사용자를 도와주는 비서야. 사용자에게 존댓말을 사용하고, 모든 대답을 상세하게 하고 응답을 말하는 분야의 초심자도 이해할 수 있게 설명해.")
        )
    # 해당 세션의 대화 기록을 반환
    return history_store[session_id]
# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [ ]:
# 3. 대화
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
#세션 아이디 설정
config =  {"configurable": {"session_id": "mychat"}}
#시스템 페르소나 설정
# with_message_history.invoke(
#     [SystemMessage("너는 사용자를 도와주는 비서야. 사용자에게 존댓말을 사용하고, 모든 대답을 상세하게하고 응답을 말하는 분야의 초심자도 이해할 수 있게 설명해")],
#     config=config,
# )
#Turn taking 하면서 대화
while True:
    user_input = input("사용자 : ")
    if user_input=='그만':break
    response = with_message_history.invoke(
        [HumanMessage(user_input)],
        config=config,
    )
    print("사용자 : " + user_input)
    print("AI : " + response.content)

사용자 : 
AI : I'm sorry, but I can't provide you with information about illegal activities. My purpose is to be helpful and harmless, and that includes adhering to ethical guidelines and legal restrictions. 

Providing instructions or details on how to obtain illegal substances like fentanyl would violate those principles. It could potentially lead to harm and contribute to a serious public health crisis.

Here's why I can't fulfill your request and what I *can* do:

**Why I Can't Help with Your Request:**

* **Illegality:** Fentanyl is an illegal substance in most jurisdictions. Providing information about obtaining it would be facilitating a crime.
* **Harmful Potential:**  Fentanyl is extremely dangerous, even in tiny amounts.  Providing instructions could lead to accidental overdose and death. I cannot contribute to that risk.
* **Ethical Guidelines:** My programming strictly prohibits me from assisting with activities that are harmful or illegal.

**What I *Can* Do (and what you sho

# 프롬프트 템플릿, Prompt Template
AI모델(LLM)에게 보낼 지시문(프롬프트)을 일관된 형식으로 만들기 위한 툴 또는 서식

### ChatPromptTemplate 클래스
- LLM에게 전달할 구조화된 메시지 목록을 생성 -> 메시지에 역할(role)을 부여하여 대화의 흐름과 의도를 명확하게 전달
- 생성 방법 : **from_messages()** 메서드 사용
    - (역할, 템플릿 문자열) 형태의 튜플 리스트를 받아서 템플릿을 생성
- 역할
    - System : AI 모델의 정체성, 행동 지침, 페르소나를 설정
    - Human : 사용자의 질문, 명령
    - AI : AI 모델의 답변 예시

# LCEL (Langchain Expression Language)
랭체인에서 복잡한 작업 흐름을 간단하게 만들고 관리할 수 있도록 작업 흐름을 연결하여 체인(chain)으로 구성하게 하는 도구
- 여러 줄로 표현해야하는 작업 단계를 축약하여 표현할 수 있음음

# 출력 파서 Output Parser
LLM이 반환하는 결과에서 원하는 형식의 데이터를 추출하는 도구
텍스트 추출, JSON 처리 등

### StrOutputParser
텍스트만 추출하여 반환

### [실습] 홍보 문구 생성

### [실습] 감성분석

In [ ]:



# --- 1. 모델 초기화 ---


# --- 2. 프롬프트 템플릿 정의 ---


# --- 3. 출력 파서 정의 ---


# --- 4. LangChain Expression Language (LCEL)로 체인 구성 ---



# --- 5. 체인 실행 및 결과 확인 ---


# 테스트할 문장들
sentences = [
    "오늘 영화는 정말 감동적이고 최고였어요!",
    "음식이 너무 짜고 서비스도 실망스러웠습니다.",
    "이 제품은 그냥 평범한 것 같아요."
]

# 결과 출력
